In [2]:
import torch
import torch_geometric
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data

# Define a simple graph with 5 nodes and their features
x = torch.tensor([[0.2], [0.5], [0.8], [0.6], [0.3]], dtype=torch.float)

# Define the edges in the graph (as an adjacency list)
edge_index = torch.tensor([[0, 1, 0, 2, 1, 2, 2, 3, 3, 4],
                           [1, 0, 2, 0, 2, 1, 3, 2, 4, 3]], dtype=torch.long)

# The edge_index tensor you've provided is a representation of the edges in a graph using a PyTorch tensor. 
# This particular edge_index is a 2D tensor with two rows and ten columns, and it defines the relationships 
# between nodes in the graph. Each column in this tensor represents an edge, with the two rows specifying the 
# source and target nodes of that edge.
y = torch.tensor([0, 1, 0, 1, 1], dtype=torch.long)

In [3]:
# Create a PyTorch Geometric Data object
data = Data(x=x, edge_index=edge_index, y=y)

In [44]:
data.x,data.y,data.x.shape

(tensor([[0.2000],
         [0.5000],
         [0.8000],
         [0.6000],
         [0.3000]]),
 tensor([0, 1, 0, 1, 1]),
 torch.Size([5, 1]))

In [40]:
# Define a simple Graph Neural Network (GNN) model
class GNNModel(nn.Module):
    def __init__(self):
        super(GNNModel, self).__init__()
        self.conv1 = torch_geometric.nn.GCNConv(1,10) #equivalent to creating input features with 1D and output features 5D  # https://www.youtube.com/watch?v=2KRAOZIULzw # answers the question how a 1D vector becomes a 5D and 5D becomes 2D
        self.conv2 = torch_geometric.nn.GCNConv(10,3)
        self.conv3 = torch_geometric.nn.GCNConv(3,1)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        x = torch.relu(x)
        x = self.conv3(x, edge_index)
        x = torch.sigmoid(x)
        return x

# Instantiate the GNN model
model = GNNModel()

# Define loss and optimizer
# criterion = torch.nn.CrossEntropyLoss()
criterion=torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Training loop
for epoch in range(100):
    optimizer.zero_grad()
    out = model(data)
    # loss = criterion(out, data.y)
    target = data.y.view(-1, 1).float()
    loss = criterion(out, target)
    loss.backward()
    optimizer.step()

# Make predictions
predictions = model(data)

In [58]:
with torch.no_grad():
    model.eval()  # Set the model to evaluation mode
    predictions = model(data)
    print(predictions)
# Apply sigmoid activation to get probabilities
predicted_probs = torch.sigmoid(predictions)

# Convert probabilities to binary predictions (0 or 1)
predicted_labels = (predicted_probs > 0.5).float()

print("Predicted Probabilities:", predicted_probs)
print("Predicted Labels:", predicted_labels)

tensor([[0.3614],
        [0.3614],
        [0.3115],
        [0.3451],
        [0.4012]])
Predicted Probabilities: tensor([[0.5894],
        [0.5894],
        [0.5773],
        [0.5854],
        [0.5990]])
Predicted Labels: tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.]])


# Prediction

In [56]:
predictions = model(data)
predictions

tensor([[0.3614],
        [0.3614],
        [0.3115],
        [0.3451],
        [0.4012]], grad_fn=<SigmoidBackward0>)

In [60]:
from torch_geometric.data import Data, Batch
x1 = torch.tensor([[0.2], [0.5], [0.8]], dtype=torch.float)
edge_index1 = torch.tensor([[0, 1, 1, 2], [1, 0, 2, 1]], dtype=torch.long)
y1 = torch.tensor([0, 1, 0], dtype=torch.long)

x2 = torch.tensor([[0.6], [0.3], [0.9]], dtype=torch.float)
edge_index2 = torch.tensor([[0, 1, 2], [1, 0, 1]], dtype=torch.long)
y2 = torch.tensor([1, 0, 1], dtype=torch.long)

# Create Data objects for each graph
data1 = Data(x=x1, edge_index=edge_index1, y=y1)
data2 = Data(x=x2, edge_index=edge_index2, y=y2)

# Batch the graphs together
batch = Batch.from_data_list([data1, data2])

In [61]:
batch

DataBatch(x=[6, 1], edge_index=[2, 7], y=[6], batch=[6], ptr=[3])